In [1]:
import numpy as np
import torch
from torch import nn
import torch.autograd.functional as F
STATELEN = 10
ACTLEN = 10
STEP_SIZE = 4

In [2]:
class Dynamics(nn.Module):

    def __init__(self, input_size, hidden_size, output_size):
        super(Dynamics, self).__init__()
        # self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(input_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, output_size),
            nn.Tanh()
        )
    def forward(self, input_element):
        output = self.linear_relu_stack(input_element)
        return output

In [3]:
class reward(nn.Module):

    def __init__(self, input_size, hidden_size, output_size = 1):
        super(reward, self).__init__()
        # self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(input_size, hidden_size),
            nn.Tanh(),
            nn.Linear(hidden_size, hidden_size),
            nn.Tanh(),
            nn.Linear(hidden_size, output_size),
            nn.Tanh()
        )

    def forward(self, input_element):
        output = self.linear_relu_stack(input_element)
        return output

In [4]:
def forward_step(state_col, actions, K_arr, k_arr, time_step):
    state = state_col[0]
    state_collection = torch.zeros((time_step, 1, STATELEN))
    reward_collection = torch.zeros((time_step, 1, 1))
    act_collection = torch.zeros((time_step, 1, ACTLEN))
    
    i = 0
    while i < time_step:
        state_collection[i] = state
        actions[i] = (torch.matmul(state_col[i] - state,torch.transpose((K_arr[i]),0,1)) + 
                      k_arr[i] + actions[i]
                     )
        
        act_collection[i] = actions[i]
        sa_input = torch.cat((state,actions[i]),dim = 1)
        #sa_input shape = [1,state_size + action_size]
        state = my_Dyna(sa_input)
        #state shape = [1,state_size]
        reward = my_reward(sa_input)
        reward_collection[i] = reward
        i = i + 1
    
    return state_collection, act_collection, reward_collection

def backward(state_col,action_col,reward_col,step_size,ifconv):

    i = step_size -1
    K_arr = torch.zeros(step_size,ACTLEN,STATELEN)
    k_arr = torch.zeros(step_size,1,ACTLEN)
    
    while i > -1:
        temp_input = torch.cat((state_col[i],act_col[i]),dim = 1)
        C_t = F.hessian(my_reward, temp_input.view(-1))
        #shape = [state+action, state+action]
        #print(torch.sum(C_t))
        F_t = F.jacobian(my_Dyna, temp_input.view(-1))
        #shape = [state, state+action]
        #print(torch.sum(F_t))
        c_t = F.jacobian(my_reward, temp_input.view(-1))
        #shape = [1, state+action]
        #print(torch.sum(c_t))
        
        if i == step_size-1:
            Q_t = C_t
            q_t = c_t
        else:
            Q_t = C_t + torch.matmul(torch.matmul(torch.transpose(F_t,0,1),V_t),F_t)
            q_t = c_t + torch.matmul(v_t,F_t)
            
        Q_pre1 = torch.split(Q_t,[STATELEN,ACTLEN])[0]
        Q_pre2 = torch.split(Q_t,[STATELEN,ACTLEN])[1]
        Q_xx = torch.split(Q_pre1,[STATELEN,ACTLEN],dim = 1)[0]
        Q_xu = torch.split(Q_pre1,[STATELEN,ACTLEN],dim = 1)[1]
        Q_uu = torch.split(Q_pre2,[STATELEN,ACTLEN],dim = 1)[1]
        
        try:
            invQuu = torch.linalg.inv(Q_uu - torch.eye(ACTLEN)) #regularize term
        except:
            invQuu = torch.linalg.inv(Q_uu + torch.eye(ACTLEN)*0.01)
            ifconv = 1
            
        K_t = -torch.matmul(invQuu, torch.transpose(Q_xu,0,1))
        transK_t = torch.transpose(K_t,0,1)
        #K_t shape = [actlen, statelen]
        q_t = torch.split(q_t, [STATELEN, ACTLEN], dim = 1)
        k_t = -torch.matmul(q_t[1],invQuu)
        #k_t shape = [1,actlen]
        
        V_t = (Q_xx + torch.matmul(Q_xu,K_t)*2 + 
               torch.matmul(torch.matmul(transK_t,Q_uu),K_t)
              )
        #V_t shape = [statelen, statelen]

        v_t = (q_t[0] + torch.matmul(k_t,torch.transpose(Q_xu,0,1)) + 
               torch.matmul(q_t[1],K_t) + 
               torch.matmul(k_t, torch.matmul(Q_uu,K_t)) 
              )
        #v_t shape = [1, statelen]
        K_arr[i] = K_t
        k_arr[i] = k_t
        i = i - 1
        
    return K_arr, k_arr, ifconv

In [5]:
my_Dyna = Dynamics(STATELEN + ACTLEN, STATELEN, STATELEN)
my_reward = reward(STATELEN + ACTLEN, STATELEN , 1)

In [6]:
state_col = torch.zeros((STEP_SIZE, 1, STATELEN))
state_col[0] = torch.rand((1, STATELEN))
act_col = torch.rand((STEP_SIZE, 1, ACTLEN))
K_arr = torch.zeros(STEP_SIZE, ACTLEN, STATELEN)
k_arr = torch.zeros(STEP_SIZE, 1, ACTLEN)
ifconv = 0

i = 0
while i < 1000:
    print(i)
    state_col, act_col ,reward_col = forward_step(state_col, act_col, K_arr, k_arr, STEP_SIZE)
    K_arr, k_arr, ifconv = backward(state_col, act_col, reward_col, STEP_SIZE, ifconv)
    print(act_col[0])
    if ifconv == 1:
        break
    
    i = i + 1


0
tensor([[0.0623, 0.8762, 0.3597, 0.4830, 0.6265, 0.7074, 0.1983, 0.2911, 0.4916,
         0.7870]], grad_fn=<SelectBackward0>)
1
tensor([[0.1125, 0.8789, 0.3399, 0.4532, 0.6035, 0.7323, 0.1395, 0.3091, 0.5292,
         0.8205]], grad_fn=<SelectBackward0>)
2
tensor([[0.1619, 0.8806, 0.3199, 0.4239, 0.5800, 0.7568, 0.0817, 0.3261, 0.5664,
         0.8539]], grad_fn=<SelectBackward0>)
3
tensor([[0.2105, 0.8813, 0.2998, 0.3950, 0.5561, 0.7810, 0.0250, 0.3421, 0.6030,
         0.8872]], grad_fn=<SelectBackward0>)
4
tensor([[ 0.2582,  0.8810,  0.2796,  0.3667,  0.5319,  0.8049, -0.0306,  0.3570,
          0.6390,  0.9203]], grad_fn=<SelectBackward0>)
5
tensor([[ 0.3050,  0.8798,  0.2594,  0.3390,  0.5073,  0.8284, -0.0850,  0.3708,
          0.6743,  0.9532]], grad_fn=<SelectBackward0>)
6
tensor([[ 0.3503,  0.8773,  0.2390,  0.3120,  0.4825,  0.8510, -0.1380,  0.3837,
          0.7089,  0.9853]], grad_fn=<SelectBackward0>)
7
tensor([[ 0.3946,  0.8739,  0.2186,  0.2856,  0.4575,  0.8732, -0

tensor([[ 1.7399,  0.2665, -0.7065, -0.3776, -0.6973,  1.6064, -1.7270,  0.3956,
          1.8312,  1.9456]], grad_fn=<SelectBackward0>)
61
tensor([[ 1.7542,  0.2531, -0.7202, -0.3824, -0.7129,  1.6166, -1.7427,  0.3933,
          1.8415,  1.9540]], grad_fn=<SelectBackward0>)
62
tensor([[ 1.7682,  0.2398, -0.7339, -0.3870, -0.7284,  1.6267, -1.7581,  0.3910,
          1.8515,  1.9621]], grad_fn=<SelectBackward0>)
63
tensor([[ 1.7821,  0.2264, -0.7475, -0.3914, -0.7437,  1.6367, -1.7732,  0.3888,
          1.8614,  1.9701]], grad_fn=<SelectBackward0>)
64
tensor([[ 1.7957,  0.2131, -0.7611, -0.3958, -0.7588,  1.6468, -1.7881,  0.3866,
          1.8711,  1.9779]], grad_fn=<SelectBackward0>)
65
tensor([[ 1.8083,  0.1997, -0.7753, -0.3999, -0.7744,  1.6569, -1.8028,  0.3835,
          1.8806,  1.9856]], grad_fn=<SelectBackward0>)
66
tensor([[ 1.8206,  0.1863, -0.7895, -0.4040, -0.7898,  1.6671, -1.8174,  0.3805,
          1.8900,  1.9930]], grad_fn=<SelectBackward0>)
67
tensor([[ 1.8328,  0

tensor([[ 2.2581, -0.4807, -1.4302, -0.5105, -1.3832,  2.1617, -2.3792,  0.2874,
          2.1827,  2.2431]], grad_fn=<SelectBackward0>)
120
tensor([[ 2.2635, -0.4924, -1.4406, -0.5113, -1.3917,  2.1705, -2.3872,  0.2868,
          2.1858,  2.2460]], grad_fn=<SelectBackward0>)
121
tensor([[ 2.2690, -0.5042, -1.4510, -0.5122, -1.4001,  2.1792, -2.3952,  0.2862,
          2.1888,  2.2489]], grad_fn=<SelectBackward0>)
122
tensor([[ 2.2743, -0.5158, -1.4613, -0.5130, -1.4084,  2.1879, -2.4031,  0.2857,
          2.1918,  2.2517]], grad_fn=<SelectBackward0>)
123
tensor([[ 2.2796, -0.5275, -1.4715, -0.5138, -1.4166,  2.1966, -2.4109,  0.2852,
          2.1948,  2.2544]], grad_fn=<SelectBackward0>)
124
tensor([[ 2.2848, -0.5391, -1.4817, -0.5146, -1.4248,  2.2053, -2.4187,  0.2847,
          2.1977,  2.2572]], grad_fn=<SelectBackward0>)
125
tensor([[ 2.2899, -0.5507, -1.4919, -0.5154, -1.4330,  2.2139, -2.4264,  0.2843,
          2.2005,  2.2599]], grad_fn=<SelectBackward0>)
126
tensor([[ 2.2

tensor([[ 2.4882, -1.1421, -1.9549, -0.5395, -1.7992,  2.6884, -2.7977,  0.3188,
          2.2857,  2.3731]], grad_fn=<SelectBackward0>)
182
tensor([[ 2.4905, -1.1519, -1.9619, -0.5400, -1.8050,  2.6963, -2.8033,  0.3203,
          2.2867,  2.3743]], grad_fn=<SelectBackward0>)
183
tensor([[ 2.4928, -1.1617, -1.9687, -0.5405, -1.8108,  2.7041, -2.8089,  0.3218,
          2.2878,  2.3756]], grad_fn=<SelectBackward0>)
184
tensor([[ 2.4951, -1.1715, -1.9756, -0.5410, -1.8166,  2.7120, -2.8145,  0.3234,
          2.2888,  2.3768]], grad_fn=<SelectBackward0>)
185
tensor([[ 2.4974, -1.1812, -1.9824, -0.5415, -1.8224,  2.7198, -2.8200,  0.3249,
          2.2898,  2.3780]], grad_fn=<SelectBackward0>)
186
tensor([[ 2.4996, -1.1910, -1.9891, -0.5420, -1.8281,  2.7276, -2.8255,  0.3266,
          2.2909,  2.3791]], grad_fn=<SelectBackward0>)
187
tensor([[ 2.5018, -1.2007, -1.9959, -0.5425, -1.8339,  2.7354, -2.8310,  0.3282,
          2.2919,  2.3803]], grad_fn=<SelectBackward0>)
188
tensor([[ 2.5

tensor([[ 2.5794, -1.7076, -2.3089, -0.5844, -2.1380,  3.1322, -3.0925,  0.4563,
          2.3502,  2.4066]], grad_fn=<SelectBackward0>)
243
tensor([[ 2.5802, -1.7164, -2.3136, -0.5854, -2.1434,  3.1389, -3.0966,  0.4592,
          2.3513,  2.4065]], grad_fn=<SelectBackward0>)
244
tensor([[ 2.5809, -1.7252, -2.3184, -0.5864, -2.1488,  3.1456, -3.1007,  0.4622,
          2.3525,  2.4063]], grad_fn=<SelectBackward0>)
245
tensor([[ 2.5816, -1.7339, -2.3231, -0.5874, -2.1542,  3.1523, -3.1048,  0.4652,
          2.3536,  2.4061]], grad_fn=<SelectBackward0>)
246
tensor([[ 2.5822, -1.7427, -2.3278, -0.5884, -2.1596,  3.1590, -3.1088,  0.4682,
          2.3548,  2.4059]], grad_fn=<SelectBackward0>)
247
tensor([[ 2.5829, -1.7514, -2.3324, -0.5894, -2.1650,  3.1656, -3.1129,  0.4712,
          2.3560,  2.4057]], grad_fn=<SelectBackward0>)
248
tensor([[ 2.5835, -1.7602, -2.3370, -0.5904, -2.1704,  3.1723, -3.1169,  0.4742,
          2.3572,  2.4054]], grad_fn=<SelectBackward0>)
249
tensor([[ 2.5

...

tensor([[ 2.3688, -4.7103, -3.7563, -1.1501, -4.3287,  5.2179, -4.4820,  2.2416,
          3.0144,  1.5960]], grad_fn=<SelectBackward0>)
732
tensor([[ 2.3685, -4.7136, -3.7585, -1.1508, -4.3320,  5.2209, -4.4849,  2.2439,
          3.0150,  1.5949]], grad_fn=<SelectBackward0>)
733
tensor([[ 2.3683, -4.7168, -3.7608, -1.1516, -4.3352,  5.2239, -4.4877,  2.2462,
          3.0155,  1.5938]], grad_fn=<SelectBackward0>)
734
tensor([[ 2.3680, -4.7200, -3.7630, -1.1524, -4.3384,  5.2269, -4.4906,  2.2485,
          3.0160,  1.5927]], grad_fn=<SelectBackward0>)
735
tensor([[ 2.3678, -4.7232, -3.7652, -1.1531, -4.3416,  5.2299, -4.4934,  2.2508,
          3.0165,  1.5916]], grad_fn=<SelectBackward0>)
736
tensor([[ 2.3675, -4.7265, -3.7675, -1.1539, -4.3448,  5.2329, -4.4963,  2.2531,
          3.0170,  1.5905]], grad_fn=<SelectBackward0>)
737
tensor([[ 2.3673, -4.7297, -3.7697, -1.1547, -4.3480,  5.2359, -4.4992,  2.2554,
          3.0175,  1.5894]], grad_fn=<SelectBackward0>)
738
tensor([[ 2.3

tensor([[ 2.3563, -4.8967, -3.8888, -1.1938, -4.5204,  5.3953, -4.6536,  2.3762,
          3.0433,  1.5305]], grad_fn=<SelectBackward0>)
793
tensor([[ 2.3561, -4.8996, -3.8909, -1.1945, -4.5234,  5.3981, -4.6563,  2.3783,
          3.0437,  1.5295]], grad_fn=<SelectBackward0>)
794
tensor([[ 2.3559, -4.9025, -3.8930, -1.1952, -4.5265,  5.4009, -4.6591,  2.3804,
          3.0441,  1.5285]], grad_fn=<SelectBackward0>)
795
tensor([[ 2.3558, -4.9053, -3.8951, -1.1958, -4.5296,  5.4037, -4.6619,  2.3825,
          3.0445,  1.5274]], grad_fn=<SelectBackward0>)
796
tensor([[ 2.3556, -4.9082, -3.8972, -1.1965, -4.5326,  5.4064, -4.6646,  2.3846,
          3.0450,  1.5264]], grad_fn=<SelectBackward0>)
797
tensor([[ 2.3555, -4.9111, -3.8993, -1.1972, -4.5357,  5.4092, -4.6674,  2.3866,
          3.0454,  1.5254]], grad_fn=<SelectBackward0>)
798
tensor([[ 2.3553, -4.9139, -3.9014, -1.1978, -4.5387,  5.4120, -4.6701,  2.3887,
          3.0458,  1.5243]], grad_fn=<SelectBackward0>)
799
tensor([[ 2.3

tensor([[ 2.3490, -5.0610, -4.0113, -1.2312, -4.7005,  5.5580, -4.8162,  2.4967,
          3.0663,  1.4698]], grad_fn=<SelectBackward0>)
853
tensor([[ 2.3490, -5.0636, -4.0133, -1.2318, -4.7035,  5.5606, -4.8188,  2.4986,
          3.0667,  1.4688]], grad_fn=<SelectBackward0>)
854
tensor([[ 2.3489, -5.0661, -4.0153, -1.2323, -4.7064,  5.5632, -4.8215,  2.5005,
          3.0670,  1.4678]], grad_fn=<SelectBackward0>)
855
tensor([[ 2.3488, -5.0687, -4.0172, -1.2329, -4.7093,  5.5659, -4.8242,  2.5024,
          3.0674,  1.4668]], grad_fn=<SelectBackward0>)
856
tensor([[ 2.3487, -5.0713, -4.0192, -1.2335, -4.7123,  5.5685, -4.8268,  2.5044,
          3.0677,  1.4659]], grad_fn=<SelectBackward0>)
857
tensor([[ 2.3486, -5.0739, -4.0212, -1.2341, -4.7152,  5.5711, -4.8295,  2.5063,
          3.0681,  1.4649]], grad_fn=<SelectBackward0>)
858
tensor([[ 2.3486, -5.0765, -4.0232, -1.2346, -4.7181,  5.5737, -4.8321,  2.5082,
          3.0684,  1.4639]], grad_fn=<SelectBackward0>)
859
tensor([[ 2.3

tensor([[ 2.3460, -5.2098, -4.1262, -1.2634, -4.8733,  5.7108, -4.9728,  2.6074,
          3.0851,  1.4123]], grad_fn=<SelectBackward0>)
913
tensor([[ 2.3460, -5.2122, -4.1281, -1.2639, -4.8762,  5.7132, -4.9753,  2.6091,
          3.0854,  1.4113]], grad_fn=<SelectBackward0>)
914
tensor([[ 2.3460, -5.2146, -4.1299, -1.2644, -4.8790,  5.7157, -4.9779,  2.6109,
          3.0857,  1.4104]], grad_fn=<SelectBackward0>)
915
tensor([[ 2.3459, -5.2169, -4.1318, -1.2649, -4.8818,  5.7182, -4.9804,  2.6127,
          3.0860,  1.4095]], grad_fn=<SelectBackward0>)
916
tensor([[ 2.3459, -5.2193, -4.1336, -1.2654, -4.8846,  5.7206, -4.9830,  2.6144,
          3.0863,  1.4085]], grad_fn=<SelectBackward0>)
917
tensor([[ 2.3459, -5.2216, -4.1355, -1.2659, -4.8874,  5.7231, -4.9855,  2.6162,
          3.0865,  1.4076]], grad_fn=<SelectBackward0>)
918
tensor([[ 2.3459, -5.2240, -4.1373, -1.2664, -4.8903,  5.7255, -4.9881,  2.6179,
          3.0868,  1.4067]], grad_fn=<SelectBackward0>)
919
tensor([[ 2.3

tensor([[ 2.3464, -5.3504, -4.2373, -1.2919, -5.0449,  5.8596, -5.1285,  2.7130,
          3.1009,  1.3559]], grad_fn=<SelectBackward0>)
975
tensor([[ 2.3465, -5.3526, -4.2390, -1.2923, -5.0476,  5.8619, -5.1310,  2.7146,
          3.1011,  1.3550]], grad_fn=<SelectBackward0>)
976
tensor([[ 2.3465, -5.3548, -4.2408, -1.2927, -5.0503,  5.8642, -5.1334,  2.7162,
          3.1013,  1.3541]], grad_fn=<SelectBackward0>)
977
tensor([[ 2.3465, -5.3569, -4.2425, -1.2931, -5.0531,  5.8665, -5.1359,  2.7179,
          3.1016,  1.3533]], grad_fn=<SelectBackward0>)
978
tensor([[ 2.3466, -5.3591, -4.2442, -1.2935, -5.0558,  5.8689, -5.1384,  2.7195,
          3.1018,  1.3524]], grad_fn=<SelectBackward0>)
979
tensor([[ 2.3466, -5.3612, -4.2459, -1.2940, -5.0585,  5.8712, -5.1408,  2.7211,
          3.1020,  1.3515]], grad_fn=<SelectBackward0>)
980
tensor([[ 2.3466, -5.3634, -4.2476, -1.2944, -5.0612,  5.8735, -5.1433,  2.7228,
          3.1022,  1.3506]], grad_fn=<SelectBackward0>)
981
tensor([[ 2.3

In [7]:
#for param in rew.parameters():
#    print(param)